In [2]:
import pandas as pd

In [4]:
#read in hospital data
df = pd.read_stata('Hospital_Inpatient.dta')
df.head(3)

,DUID,PID,DUPERSID,EVNTIDX,EVENTRN,ERHEVIDX,FFEEIDX,PANEL,MPCDATA,IPBEGYR,...,IPDOF23X,IPDSL23X,IPDWC23X,IPDOT23X,IPDXP23X,IPDTC23X,IMPFLAG,PERWT23F,VARSTR,VARPSU
0,2790023,102,2790023102,2790023102002001,3 ROUND 3,-1,-1,27 PANEL 27,1 HAS MPC DATA,2022,...,0.0,0.0,0.0,0.0,1126.52,18923.0,2 COMPLETE MPC DATA,19699.576310,2037,1
1,2790046,103,2790046103,2790046103004701,3 ROUND 3,2790046103004801,-1,27 PANEL 27,2 NO MPC DATA,2023,...,0.0,0.0,0.0,0.0,428.95,1464.0,4 PARTIALLY IMPUTED,9131.375404,2055,3
2,2790046,103,2790046103,2790046103004901,3 ROUND 3,2790046103005001,-1,27 PANEL 27,1 HAS MPC DATA,2023,...,0.0,0.0,0.0,0.0,428.95,1464.0,2 COMPLETE MPC DATA,9131.375404,2055,3


In [ ]:
#check columns and dtype
df.info()

In [ ]:
#create df with columns of interest
hospital_df = df[['DUPERSID', 'EVNTIDX', 'IPENDMM', 'IPDMR23X','IPDMD23X','IPDPV23X',
                  'IPFMD23X','IPFMR23X','IPFPV23X', 'EVENTRN']].copy()
hospital_df.head(3)


In [ ]:
#change column names from codebook variables
hospital_df.rename(columns={'DUPERSID': 'person_id', 'EVNTIDX': 'visit_id', 'IPENDMM': 'month', 'IPDMR23X': 'dr_medicare',
                            'IPDMD23X': 'dr_medicaid','IPDPV23X': 'dr_private',
                            'IPFMD23X': 'f_medicaid','IPFMR23X': 'f_medicare','IPFPV23X': 'f_private', 'EVENTRN': 'round'}, 
                            inplace=True)
hospital_df.info()

In [ ]:
#sum dr and facility payments to one column for medicare, medicaid, private
hospital_df['medicaid']= hospital_df['dr_medicaid'] + hospital_df['f_medicaid']
hospital_df['medicare']= hospital_df['dr_medicare'] + hospital_df['f_medicare']
hospital_df['private'] = hospital_df['dr_private'] + hospital_df['f_private']

#drop old columns
hospital_df = hospital_df.drop(['dr_medicaid', 'f_medicaid', 'dr_medicare', 'f_medicare', 'dr_private', 'f_private'], axis=1)
hospital_df.head()

In [ ]:
#check category datatype in month column
hospital_df['month'].unique()

[1, 2, 5, 10, 12, ..., 3, 8, 7, 6, '-8 DON'T KNOW']
Length: 13
Categories (13, object): ['-8 DON'T KNOW' < 1 < 2 < 3 ... 9 < 10 < 11 < 12]

In [15]:
#drop invalid months
hospital_df = hospital_df.dropna(subset=['month'])

#change month datatype
#category label to string, coerce to NANs
hospital_df['month'] = pd.to_numeric(hospital_df['month'].astype(str), errors='coerce')
#drop Nans
hospital_df = hospital_df.dropna(subset=['month'])
#change datatype to int
hospital_df['month'] = hospital_df['month'].astype(int)

hospital_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1819 entries, 0 to 1829
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype   
---  ------     --------------  -----   
 0   person_id  1819 non-null   object  
 1   visit_id   1819 non-null   object  
 2   month      1819 non-null   int64   
 3   round      1819 non-null   category
 4   medicaid   1819 non-null   float64 
 5   medicare   1819 non-null   float64 
 6   private    1819 non-null   float64 
dtypes: category(1), float64(3), int64(1), object(2)
memory usage: 101.5+ KB
